## 介紹

本課程將涵蓋：
- 什麼是函式呼叫及其應用情境
- 如何使用 OpenAI 建立函式呼叫
- 如何將函式呼叫整合到應用程式中

## 學習目標

完成本課程後，你將能夠理解並學會：

- 使用函式呼叫的目的
- 使用 OpenAI 服務設定函式呼叫
- 為你的應用情境設計有效的函式呼叫


## 了解函式呼叫

在這堂課中，我們要為我們的教育新創公司打造一個功能，讓使用者可以透過聊天機器人來尋找技術課程。我們會根據使用者的技能等級、目前職位以及感興趣的技術來推薦合適的課程。

為了完成這個目標，我們會結合以下幾個工具：
 - 使用 `OpenAI` 來為使用者打造聊天體驗
 - 使用 `Microsoft Learn Catalog API`，根據使用者的需求協助他們找到合適的課程
 - 使用 `Function Calling`，將使用者的查詢傳送給函式，進而發送 API 請求

首先，讓我們來看看為什麼我們需要使用函式呼叫：

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 從 GPT 取得新的回應，這次它可以看到函式的回傳結果


print(second_response.choices[0].message)


### 為什麼要使用 Function Calling

如果你已經完成了本課程中的其他課程，應該已經了解使用大型語言模型（LLMs）的強大之處。希望你同時也能看到它們的一些限制。

Function Calling 是 OpenAI 服務中的一項功能，旨在解決以下挑戰：

回應格式不一致：
- 在有 function calling 之前，來自大型語言模型的回應通常是非結構化且不一致的。開發人員必須撰寫複雜的驗證程式碼來處理每一種不同的輸出格式。

與外部資料整合有限：
- 在這個功能推出之前，要將應用程式其他部分的資料整合進聊天情境中是很困難的。

透過標準化回應格式並讓外部資料能夠無縫整合，function calling 簡化了開發流程，也減少了額外驗證邏輯的需求。

使用者無法獲得像是「斯德哥爾摩現在的天氣如何？」這樣的答案。這是因為模型只能存取訓練時期的資料。

讓我們來看看下面這個例子，說明這個問題：

假設我們想建立一個學生資料庫，以便能夠為他們推薦合適的課程。下面有兩個學生的描述，所包含的資料非常相似。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

我們想要將這個資料發送給 LLM 進行解析。之後可以在我們的應用程式中，將這些資料發送到 API 或儲存到資料庫。

我們來建立兩個相同的提示，指示 LLM 我們感興趣的資訊內容：


我們想將這個發送給一個大型語言模型，以解析對我們產品重要的部分。因此，我們可以創建兩個相同的提示來指示該大型語言模型：


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


在建立這兩個提示後，我們將使用 `openai.ChatCompletion` 將它們發送給 LLM。我們將提示存儲在 `messages` 變數中，並將角色設為 `user`。這是為了模擬用戶向聊天機器人發送訊息的情境。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

即使提示內容相同、描述也很類似，我們仍然可能會得到不同格式的 `Grades` 屬性。

如果你多次執行上面的 cell，格式可能會是 `3.7` 或 `3.7 GPA`。

這是因為 LLM 以書面提示的非結構化資料作為輸入，回傳的也是非結構化資料。我們需要有一個結構化的格式，這樣在儲存或使用這些資料時才知道該預期什麼。

透過使用函式呼叫（functional calling），我們可以確保收到的是結構化資料。使用函式呼叫時，LLM 並不會真的執行任何函式。相反地，我們為 LLM 設計一個回應的結構，然後利用這些結構化的回應來決定在應用程式中要執行哪個函式。


![函式呼叫流程圖](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.tw.png)


### 使用函式呼叫的應用情境

**呼叫外部工具**  
聊天機器人在回答使用者問題時表現優異。透過函式呼叫，聊天機器人可以根據使用者的訊息來完成特定任務。例如，學生可以請聊天機器人「幫我寄封信給老師，說我需要更多協助」。這時就可以呼叫 `send_email(to: string, body: string)` 這個函式。

**建立 API 或資料庫查詢**  
使用者可以用自然語言查詢資訊，系統會自動將其轉換成格式化的查詢或 API 請求。例如，老師詢問「哪些學生完成了上次的作業？」時，可以呼叫名為 `get_completed(student_name: string, assignment: int, current_status: string)` 的函式。

**建立結構化資料**  
使用者可以將一段文字或 CSV 檔案，透過 LLM 萃取出重要資訊。例如，學生可以把一篇關於和平協議的維基百科文章轉換成 AI 快閃卡。這可以透過 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` 這個函式來完成。


## 2. 建立你的第一個函式呼叫

建立函式呼叫的流程包含三個主要步驟：
1. 使用你的函式清單和使用者訊息呼叫 Chat Completions API
2. 讀取模型的回應以執行動作，例如執行函式或 API 呼叫
3. 再次呼叫 Chat Completions API，並將你的函式回應帶入，讓模型利用這些資訊回覆使用者


![函式呼叫的流程](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.tw.png)


### 函式呼叫的元素

#### 使用者輸入

第一步是建立一則使用者訊息。你可以動態地從文字輸入欄取得值，或直接在這裡指定一個值。如果你是第一次使用 Chat Completions API，我們需要定義訊息的 `role` 和 `content`。

`role` 可以是 `system`（建立規則）、`assistant`（模型）或 `user`（最終使用者）。在函式呼叫的情境下，我們會將其設為 `user`，並給一個範例問題。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 建立函式。

接下來我們會定義一個函式以及它的參數。這裡我們只會用一個叫做 `search_courses` 的函式，但你也可以建立多個函式。

**重要事項**：函式會被包含在系統訊息中傳送給 LLM，並且會佔用你可用的 token 數量。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定義說明**

函式定義結構有多層級，每一層都有自己的屬性。以下是巢狀結構的說明：

**頂層函式屬性：**

`name` -  想要呼叫的函式名稱。

`description` - 這是關於函式如何運作的說明。這裡需要具體且清楚地描述。

`parameters` - 你希望模型在回應中產生的值及其格式的列表。

**參數物件屬性：**

`type` - 參數物件的資料型別（通常是 "object"）

`properties` - 模型在回應中會用到的具體值列表

**個別參數屬性：**

`name` - 由屬性鍵隱含定義（例如 "role"、"product"、"level"）

`type` - 此特定參數的資料型別（例如 "string"、"number"、"boolean"）

`description` - 這個特定參數的說明

**選用屬性：**

`required` - 一個陣列，列出完成函式呼叫時必須提供的參數


### 呼叫函式

在定義好函式之後，接下來我們需要在呼叫 Chat Completion API 時將它包含進去。我們可以透過在請求中加入 `functions` 來達成這個目的。在這個例子中是 `functions=functions`。

另外，也可以選擇將 `function_call` 設為 `auto`。這代表我們讓 LLM 根據使用者訊息自行決定要呼叫哪個函式，而不是由我們手動指定。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

現在讓我們來看看回應內容以及其格式：

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

你可以看到這裡呼叫了函式名稱，並且從使用者訊息中，LLM 能夠找到資料來填入函式的參數。


## 3.將函式呼叫整合到應用程式中

在我們測試過 LLM 格式化的回應後，現在可以把它整合到應用程式裡。

### 管理流程

要把這個功能整合到我們的應用程式，請依照以下步驟進行：

首先，呼叫 OpenAI 服務，並將訊息儲存到名為 `response_message` 的變數中。


In [ ]:
response_message = response.choices[0].message

現在我們將定義一個函數，該函數將呼叫 Microsoft Learn API 以獲取課程列表：


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作為最佳實踐，我們接下來會檢查模型是否想要呼叫某個函式。之後，我們會建立一個可用的函式，並將其與被呼叫的函式進行配對。

接著，我們會將函式的參數與來自 LLM 的參數進行對應。

最後，我們會附加函式呼叫訊息以及 `search_courses` 訊息所回傳的值。這樣 LLM 就能獲得所有所需資訊，並以自然語言回應使用者。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 程式挑戰

做得好！如果你想繼續學習 OpenAI Function Calling，可以試著建構以下內容：https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - 增加更多函式參數，幫助學習者找到更多課程。你可以在這裡找到可用的 API 參數：
 - 建立另一個函式呼叫，讓學習者可以提供更多資訊，例如他們的母語
 - 當函式呼叫和/或 API 呼叫沒有回傳合適課程時，加入錯誤處理



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們致力於確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
